In [17]:
import redis
import configparser
import numpy as np
import pandas as pd
import tasks.models as m

from sqlalchemy import create_engine, func, distinct
from sqlalchemy.orm import Session
from sqlalchemy import tuple_

In [18]:
# Get Config
config = configparser.ConfigParser()
config.read("config/config.ini")

# Create SQLAlchemy Session
engine = create_engine(config["DB"].get("db_url"))
session = Session(engine)

# Create Redis Client
r = redis.Redis(
    host=config["REDIS"].get("host"),
    port=config["REDIS"].get("port"),
    db=config["REDIS"].get("db"),
)

In [19]:
# Get users who commented on more than two different subreddits
s = (
    session.query(m.Redditor_Subreddit_Comment.redditor_id)
    .group_by(m.Redditor_Subreddit_Comment.redditor_id)
    .having(func.count(m.Redditor_Subreddit_Comment.subreddit_id.distinct()) > 2)
    .subquery()
)
redditor_subreddit_comment = np.array(
    session.query(
        m.Redditor_Subreddit_Comment.redditor_id, m.Subreddit.name, m.Redditor.name
    )
    .distinct(m.Redditor_Subreddit_Comment.redditor_id, m.Subreddit.name)
    .filter(m.Redditor_Subreddit_Comment.redditor_id.in_(s))
    .join(m.Subreddit, m.Redditor_Subreddit_Comment.subreddit_id == m.Subreddit.id)
    .join(m.Redditor, m.Redditor_Subreddit_Comment.redditor_id == m.Redditor.id)
    .all()
).T
df = pd.DataFrame(
    {
        "user_id": redditor_subreddit_comment[0],
        "subreddit_id": redditor_subreddit_comment[1],
        "username": redditor_subreddit_comment[2],
    }
)

In [20]:
user_ids = np.unique(df['user_id'])

In [21]:
def compare_recommendation_and_interaction(user_id):
    if type(user_id) != list:
        user_id = [user_id]
    for u in user_id:
        print(f"user: {df[df['user_id']==u]['username'].iloc[0]}") 
        print(f"commented on {np.unique(df[df['user_id']==u]['subreddit_id'])}")
        print(f"recommended {list(map(lambda x: x.decode(), r.lrange(u, 0, -1)))}")
        print("")


In [22]:
compare_recommendation_and_interaction(user_ids[0])

user: arq25
commented on ['StockMarket' 'sysadmin' 'whiskey']
recommended ['pennystocks', 'bourbon', 'RobinHoodPennyStocks', 'options', 'Daytrading']



In [23]:
compare_recommendation_and_interaction(user_ids[1])

user: ac13332
commented on ['AskUK' 'britishproblems' 'golf']
recommended ['CasualUK', 'unitedkingdom', 'ukpolitics', 'london', 'UKPersonalFinance']



In [24]:
compare_recommendation_and_interaction(user_ids[2])

user: ZetTheLegendaryHero
commented on ['DCcomics' 'Doom' 'batman' 'goodanimemes']
recommended ['DC_Cinematic', 'Spiderman', 'SpidermanPS4', 'raimimemes', 'starwarsmemes']



In [25]:
compare_recommendation_and_interaction(user_ids[3])

user: M6D_Magnum
commented on ['MilitaryPorn' 'assassinscreed' 'gundeals' 'guns']
recommended ['ar15', 'GunPorn', 'progun', 'EDC', 'knives']



In [26]:
compare_recommendation_and_interaction(user_ids[4])

user: Eggith
commented on ['askgaybros' 'dbz' 'tf2']
recommended ['gay', 'gaybros', 'gay_irl', 'DBZDokkanBattle', 'gaymers']



In [27]:
compare_recommendation_and_interaction(user_ids[5])

user: Awesomefoxhound
commented on ['eu4' 'reddeadredemption2' 'selfimprovement']
recommended ['hoi4', 'CrusaderKings', 'RedDeadOnline', 'DecidingToBeBetter', 'imaginarymaps']



In [28]:
compare_recommendation_and_interaction(user_ids[10])

user: walnoter
commented on ['animememes' 'dndmemes' 'furry']
recommended ['wholesomeanimemes', 'goodanimemes', 'Konosuba', 'awwnime', 'MemeTemplatesOfficial']



In [29]:
compare_recommendation_and_interaction(user_ids[110])

user: los33ramos
commented on ['LosAngeles' 'PS5' 'playstation']
recommended ['lakers', 'PS4Deals', 'XboxSeriesX', 'xbox', 'PlayStationPlus']



In [30]:
compare_recommendation_and_interaction(user_ids[1000])

user: SinfullySinless
commented on ['AskAnAmerican' 'BeautyGuruChatter' 'IllegallySmolCats'
 'LeopardsAteMyFace' 'Teachers' 'TheRightCantMeme' 'fatlogic'
 'notliketheothergirls' 'thesims']
recommended ['Sims4', 'Makeup', 'rupaulsdragrace', '90DayFiance', 'TumblrInAction']



In [31]:
# compare_recommendation_and_interaction('10rndb')
# compare_recommendation_and_interaction(list(np.unique(df[df['subreddit_id']=='learnpython']['user_id'])))